In [28]:
import pandas as pd
# 抓 pkl 檔案
all_train_data = pd.read_pickle('../data/twitter/train_data.pkl')
all_test_data = pd.read_pickle('../data/twitter/test_data.pkl')

all_train_data.drop_duplicates(subset=['str'], inplace=True, keep='first')

# 查看 train_data 的 emotion 分布
check_train_emotion = all_train_data["emotion"].value_counts()
print("train:\n",check_train_emotion)
min_count = check_train_emotion.min()
# 從 train_data 對每種 emotion 都隨機抽 39867 筆出來

down_sampled_train_data = pd.DataFrame()
for emotion in all_train_data["emotion"].unique():
    sample = all_train_data[all_train_data["emotion"] == emotion].sample(n=min_count, random_state=1)
    down_sampled_train_data = pd.concat([down_sampled_train_data, sample])

# print(down_sampled_train_data.shape)
# print(down_sampled_train_data.head())
# print(down_sampled_train_data.tail())

# 發現是按照 emotion 排序的，所以要 shuffle
down_sampled_train_data = down_sampled_train_data.sample(frac=1, random_state=1)

# print(down_sampled_train_data.shape)
# print(down_sampled_train_data.head())
# print(down_sampled_train_data.tail())

down_sampled_train_data.reset_index(drop=True, inplace=True)
print(down_sampled_train_data.head())
print(down_sampled_train_data.tail())

check_train_emotion = down_sampled_train_data["emotion"].value_counts()
print("train:\n",check_train_emotion)


train:
 joy             514266
anticipation    248695
trust           204885
sadness         193198
disgust         138996
fear             63821
surprise         48205
anger            39712
Name: emotion, dtype: int64
   tweet_id                                                str  \
0  0x2f3224  ‘[FREE] Epic Pop Hip Hop Beat - Crazy Rap R&B ...   
1  0x21c86e                       @luvselefantes I know ❤ <LH>   
2  0x2700e9                           <LH> is the #HardestPart   
3  0x34860d  This baby daddy is such a piece of shit. <LH> ...   
4  0x1e65d7  Thousands of miserable blistering iconoclasts ...   

                                                text       emotion  
0  [‘[FREE] Epic Pop Hip Hop Beat - Crazy Rap R&B...      surprise  
1                     [@luvselefantes I know ❤ <LH>]         trust  
2                         [<LH> is the #HardestPart]  anticipation  
3  [This baby daddy is such a piece of shit. <LH>...      surprise  
4  [Thousands of miserable blistering 

In [29]:
# 切 data train -> train and valid include x and y
# 切 data test -> test include x and y

from sklearn.model_selection import train_test_split

train_data, valid_data = train_test_split(down_sampled_train_data, test_size=0.2, random_state=1)
print(train_data.shape)
print(valid_data.shape)

train_data.reset_index(drop=True, inplace=True)
valid_data.reset_index(drop=True, inplace=True)





(254156, 4)
(63540, 4)


In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from scipy.sparse import csr_matrix
import tensorflow as tf
import numpy as np
nltk.download('punkt')

# TF -IDF 特徵
TF_IDF = TfidfVectorizer(max_features=10000, tokenizer=nltk.word_tokenize)
TF_IDF.fit(train_data["str"])

train_data_TF_IDF = TF_IDF.transform(train_data["str"])

print(train_data_TF_IDF.shape)


c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(254156, 10000)


In [34]:
feature_names = TF_IDF.get_feature_names()
print(feature_names[9990:10000])

['🤬', '🤷\u200d♀️', '🤷🏻\u200d♀️', '🤷🏼\u200d♀️', '🤷🏽\u200d♀️', '🤷🏽\u200d♂️', '🤷🏾\u200d♀️', '🤷🏾\u200d♂️', '🦃', '🦋']


c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [45]:
def convert_to_sparse_tensor(csr_mat):
    coo = csr_mat.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data.astype(np.float32), coo.shape)

x_train = TF_IDF.transform(train_data["str"])
x_train = convert_to_sparse_tensor(x_train)
x_train = tf.sparse.reorder(x_train)


y_train = train_data["emotion"]

x_valid = TF_IDF.transform(valid_data["str"])
x_valid = convert_to_sparse_tensor(x_valid)
x_valid = tf.sparse.reorder(x_valid)
y_valid = valid_data["emotion"]

x_test = TF_IDF.transform(all_test_data["str"])
x_test = convert_to_sparse_tensor(x_test)
x_test = tf.sparse.reorder(x_test)

print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)
print(x_test.shape)





(254156, 10000)
(254156,)
(63540, 10000)
(63540,)
(411972, 10000)


In [46]:
import keras
import numpy as np
from sklearn.preprocessing import LabelEncoder

label_coder = LabelEncoder()
label_coder.fit(y_train)

def label_encode(coder,label):
    one_hot = coder.transform(label)
    return keras.utils.to_categorical(one_hot)

def label(coder,one_hot):
    predict = np.argmax(one_hot, axis=1)
    return coder.inverse_transform(predict)

print("check label_encode and label function")
print(label_coder.classes_)
print(y_train[:5])
print(y_train.shape)

y_train_one_hot = label_encode(label_coder,y_train)
y_valid_one_hot = label_encode(label_coder,y_valid)

print(y_train_one_hot.shape)
print(y_train_one_hot[:5])

check label_encode and label function
['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']
0     sadness
1       trust
2         joy
3        fear
4    surprise
Name: emotion, dtype: object
(254156,)
(254156, 8)
[[0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]]


In [ ]:
x_train_dense = tf.sparse.to_dense(x_train) if isinstance(x_train, tf.sparse.SparseTensor) else x_train
x_valid_dense = tf.sparse.to_dense(x_valid) if isinstance(x_valid, tf.sparse.SparseTensor) else x_valid
x_test_dense = tf.sparse.to_dense(x_test) if isinstance(x_test, tf.sparse.SparseTensor) else x_test
y_train_one_hot_dense = tf.sparse.to_dense(y_train_one_hot) if isinstance(y_train_one_hot, tf.sparse.SparseTensor) else y_train_one_hot
y_valid_one_hot_dense = tf.sparse.to_dense(y_valid_one_hot) if isinstance(y_valid_one_hot, tf.sparse.SparseTensor) else y_valid_one_hot

# 使用 np.save 保存数据，确保第一个参数是文件名
np.save('../data/twitter/x_train.npy', x_train_dense.numpy())
np.save('../data/twitter/x_valid.npy', x_valid_dense.numpy())
np.save('../data/twitter/x_test.npy', x_test_dense.numpy())
np.save('../data/twitter/y_train_one_hot.npy', y_train_one_hot_dense.numpy())
np.save('../data/twitter/y_valid_one_hot.npy', y_valid_one_hot_dense.numpy())

In [ ]:
# 加载 NumPy 数组
x_train_np = np.load('../data/twitter/x_train.npy')
x_valid_np = np.load('../data/twitter/x_valid.npy')
x_test_np = np.load('../data/twitter/x_test.npy')
y_train_one_hot_np = np.load('../data/twitter/y_train_one_hot.npy')
y_valid_one_hot_np = np.load('../data/twitter/y_valid_one_hot.npy')

# 将 NumPy 数组转换为 TensorFlow 张量
x_train_tensor = tf.convert_to_tensor(x_train_np, dtype=tf.float32)
x_valid_tensor = tf.convert_to_tensor(x_valid_np, dtype=tf.float32)
x_test_tensor = tf.convert_to_tensor(x_test_np, dtype=tf.float32)
y_train_one_hot_tensor = tf.convert_to_tensor(y_train_one_hot_np, dtype=tf.float32)
y_valid_one_hot_tensor = tf.convert_to_tensor(y_valid_one_hot_np, dtype=tf.float32)

# 将张量转换为稀疏张量
x_train = tf.sparse.from_dense(x_train_tensor)
x_valid = tf.sparse.from_dense(x_valid_tensor)
x_test = tf.sparse.from_dense(x_test_tensor)
y_train_one_hot = tf.sparse.from_dense(y_train_one_hot_tensor)
y_valid_one_hot = tf.sparse.from_dense(y_valid_one_hot_tensor)

In [47]:
input_shape = x_train.shape[1]
print(input_shape)

out_put_shape = len(label_coder.classes_)
print(out_put_shape)






10000
8


In [48]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax
from keras.layers import Dropout

model_input = Input(shape=(input_shape,))

layer1 = Dense(1024)(model_input)
active1 = ReLU()(layer1)

dropout1 = Dropout(0.5)(active1)

layer2 = Dense(512)(dropout1)
active2 = ReLU()(layer2)

dropout2 = Dropout(0.5)(active2)

layer3 = Dense(256)(dropout2)
active3 = ReLU()(layer3)

dropout3 = Dropout(0.5)(active3)

layer4 = Dense(128)(dropout3)
active4 = ReLU()(layer4)

dropout4 = Dropout(0.5)(active4)

layer5 = Dense(8)(dropout4)
active5 = Softmax()(layer5)

output = active5

model = Model(inputs=model_input, outputs=output)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()



Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 10000)]           0         
                                                                 
 dense_10 (Dense)            (None, 1024)              10241024  
                                                                 
 re_lu_8 (ReLU)              (None, 1024)              0         
                                                                 
 dropout_8 (Dropout)         (None, 1024)              0         
                                                                 
 dense_11 (Dense)            (None, 512)               524800    
                                                                 
 re_lu_9 (ReLU)              (None, 512)               0         
                                                                 
 dropout_9 (Dropout)         (None, 512)               0   

In [50]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger('logs/training_log.csv')
# training setting
epochs = 25
batch_size = 64

history = model.fit(x_train, y_train_one_hot, 
                    batch_size=batch_size, 
                    epochs=epochs, 
                    validation_data=(x_valid, y_valid_one_hot), 
                    callbacks=[csv_logger])


Epoch 1/25


c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_2/dense_10/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_2/dense_10/embedding_lookup_sparse/Reshape:0", shape=(None, 1024), dtype=float32), dense_shape=Tensor("gradient_tape/model_2/dense_10/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


 250/3972 [>.............................] - ETA: 5:20 - loss: 2.0253 - accuracy: 0.1688

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
# 劃出 loss 和 accuracy 的圖
plt.plot(history.history["loss"], label="train_loss")
plt.plot(history.history["val_loss"], label="valid_loss")
plt.legend()
plt.show()

plt.plot(history.history["categorical_accuracy"], label="train_accuracy")
plt.plot(history.history["val_categorical_accuracy"], label="valid_accuracy")
plt.legend()
plt.show()

In [ ]:
## predict
pred_result = model.predict(x_test, batch_size=128)
pred_result[:5]

In [ ]:
pred_result = label(label_coder, pred_result)
pred_result[:5]

In [ ]:
# 合併 x_test 回 all_test_data
all_test_data["emotion"] = pred_result
print(all_test_data.head())

# 讀 csv sample_Submission.csv, 依據 id 對應到 all_test_data 的 emotion
ans_csv = pd.read_csv('../data/twitter/sample_Submission.csv')
print(ans_csv.head())

matching_ids = ans_csv['id'].head()
matching_data = all_test_data[all_test_data['tweet_id'].isin(matching_ids)]
print(matching_data)

merged_data = pd.merge(ans_csv, matching_data[['tweet_id', 'emotion']], left_on='id', right_on='tweet_id', how='left')



# 可选择删除多余的列
merged_data.drop(['tweet_id', 'str',"text"], axis=1, inplace=True)

# 保存更改后的数据到 CSV
merged_data.to_csv('updated_csv_file.csv', index=False)

# 打印完成信息
print("\n已更新并保存 CSV 文件。")
